In [ ]:
TFile *file = new TFile("data/InvariantMass.root");

//Available types:
/*
    K0
    Lambda
    AntiLambda
*/
TString type = "K0";

//Available collisions:
/*
    pp
    pbpb
*/
TString collision = "pbpb";

//Available centralities:
/*
    000_010
    010_020
    020_030
    030_040
    040_050
    050_060
    060_070
    070_080
    090_100
*/
TString centrality = "000_010";


In [ ]:
//Find a folder with the particle type
auto type_key = file->FindKey(type);
auto obj = type_key->ReadObj();
                              
if (!obj->IsA()->InheritsFrom(TDirectory::Class())) {
    Printf("Wrong directory!");
}
auto *dir = dynamic_cast<TDirectory*>(obj);

//Find the requested histogram
TString histogram_name;
TString type_lowercase = type;
type_lowercase.ToLower();
if(collision == "pbpb") {
    histogram_name = collision + "_" + centrality + "_" + type_lowercase;
} else {
    histogram_name = collision + "_" + type_lowercase;
}

auto dir_key = dir->FindKey(histogram_name);
obj = dir_key->ReadObj();
if (!obj->IsA()->InheritsFrom(TH1::Class())) {
    Printf("Not a histogram, but %s", obj->ClassName());
}
auto histogram = dynamic_cast<TH1*>(obj);

In [ ]:
//Draw the histogram
TCanvas c1("","",1600,1200);
histogram->Draw();
c1.Draw();

In [ ]:
//Set the default background and signal ranges
Double_t sg_min = 0, sg_max = 2, bg_min = 0, bg_max = 2;

//Update ranges here
sg_min = 0.48;
sg_max = 0.51;
bg_min = 0.4;
bg_max = 0.6;

auto fit = new TF1("fit", "gausn(0)+pol2(3)", 0, 2);
fit->SetParNames("Y", "#mu", "#sigma", "A", "B", "C");
fit->SetRange(bg_min, bg_max);
fit->SetParameters(80, (sg_min + sg_max) / 2, (sg_max - sg_min) / 4);
fit->SetParLimits(0, 0, 1e9);
fit->SetParLimits(1, sg_min, sg_max);
fit->SetParLimits(2, 0, (sg_max - sg_min) / 2);
fit->SetLineColor(kGreen + 1);

auto bg = new TF1("bg", "pol2(0)", 0, 2);
bg->SetParNames("A", "B", "C");
bg->SetRange(bg_min, bg_max);

auto result = histogram->Fit(fit, "NQSR", "", bg_min, bg_max);

bg->SetLineColor(kBlue + 1);
bg->SetParameters(fit->GetParameter(3), fit->GetParameter(4), fit->GetParameter(5));
bg->SetParError(0, fit->GetParError(3));
bg->SetParError(1, fit->GetParError(4));
bg->SetParError(2, fit->GetParError(5));

auto red = new TLatex(.98, .8, Form("#chi^{2}/#nu=%6.3f", result->Chi2() / result->Ndf()));
red->SetTextAlign(32);
red->SetTextFont(42);
red->SetNDC();
histogram->GetListOfFunctions()->Add(red);

TCanvas c2("","",1600,1200);
histogram->Draw();
fit->Draw("same");
bg->Draw("same");
c2.Draw();

In [ ]:
TMatrixDSym tCov = result->GetCovarianceMatrix();
TMatrixDSym bCov(3);

for (Int_t i = 0; i < 3; i++) {
    for (Int_t j = 0; j < 3; j++) {
        bCov(i, j) = tCov(3 + i, 3 + j);
    }
}

Double_t dx = histogram->GetXaxis()->GetBinWidth(1);

auto Total = fit->Integral(sg_min, sg_max);
auto TotalE = fit->IntegralError(sg_min, sg_max, result->GetParams(), tCov.GetMatrixArray());
auto Back = bg->Integral(sg_min, sg_max);
auto BackE = bg->IntegralError(sg_min, sg_max, &(result->GetParams()[3]), bCov.GetMatrixArray());

Total /= dx;
TotalE /= dx;
Back /= dx;
BackE /= dx;

auto Signal = Total - Back;
auto SignalE = TMath::Sqrt(TotalE * TotalE + BackE * BackE);

auto Mean = fit->GetParameter(1);
auto MeanE = fit->GetParError(1);

auto Sigma = fit->GetParameter(2);
auto SigmaE = fit->GetParError(2);

In [ ]:
printf("Entries:\t%d\n", int(histogram->GetEntries()));
printf("Total:\t\t%6.0f +- %3.0f\n", Total, TotalE);
printf("Background:\t%6.0f +- %3.0f\n", Back, BackE);
printf("Signal:\t\t%6.0f +- %3.0f\n", Signal, SignalE);
printf("Mean:\t\t%6.4f +- %6.4f\n", Mean, MeanE);
printf("Sigma:\t\t%6.4f +- %6.4f\n", Sigma, SigmaE);